In [45]:
#%% Import libraries
import numpy as np
from sympy import *
# from sympy.physics.mechanics import ReferenceFrame
from IPython.display import display, Latex, Math
init_printing(use_latex='mathjax')

from sys import path
path.append(
    "c:\\Users\\diego\\Dropbox\\Academic\\MEng Space Systems\\3. DOCA\\ADCS functions")
import ADCS_Functions as adcs
import ADCS_Functions_sym as adcs_sym

___________________________________

A spacecraft with a principal axes body-fixed frame b, has corresponding 
principal moments of inertia

In [46]:
I1 = 8  # kg*m^2
I2 = 12  # kg*m^2
I3 = 10  # kg*m^2

I = np.array([[I1, 0, 0], [0, I2, 0], [0, 0, I3]])
adcs.print_matrix(I)

<IPython.core.display.Math object>

It is desired to spin the spacecraft about the principal z-axis with angular 
velocity $\omega_z = 0.1 $ rad/s. 

Assuming that the spacecraft has the momentum 
wheel with spin-axis aligned with the principal z-axis, determine the required 
relative wheel angular momentum $h_s$, to make the desired attitude motion 
passively stable under torque-free conditions and without energy dissipation

In [47]:
wz = 0.1 # rad/s

__________________

From lecture ADCS-IXB, for passive stability, we must either:

$\lambda>I_{2}$ and $\lambda>I_{1}$ 

or 

$\lambda<I_{2}$ and $\lambda<I_{1}$

where:

In [48]:
# Initialize the symbolic variables
omega1sim, omega2sim, omega3sim = symbols('\omega_1 \omega_2 \omega_3')
dotomega1sim, dotomega2sim, dotomega3sim = symbols(
    '\dot{\omega}_1 \dot{\omega}_2 \dot{\omega}_3')

I1_sym, I2_sym, I3_sym = symbols('I1, I2, I3')
hs_sym = symbols('h_s')
Omega_sym = symbols('Omega')

# Initialize the symbolic equation for lambda
lambd_sym = I3_sym + hs_sym / Omega_sym
Eq(Symbol('\lambda'), lambd_sym)


               hₛ
\lambda = I₃ + ──
               Ω 

In [49]:
# From Euler's equations
T1_sym = (I1_sym*dotomega1sim - (I2_sym - I3_sym)*omega2sim*omega3sim)
T2_sym = (I2_sym*dotomega2sim - (I3_sym - I1_sym)*omega3sim*omega1sim)
T3_sym = (I3_sym*dotomega3sim - (I1_sym - I2_sym)*omega1sim*omega2sim)

T1_sym, T2_sym, T3_sym

(I₁⋅\dot{\omega}₁ - \omega₂⋅\omega₃⋅(I₂ - I₃), I₂⋅\dot{\omega}₂ - \omega₁⋅\ome
ga₃⋅(-I₁ + I₃), I₃⋅\dot{\omega}₃ - \omega₁⋅\omega₂⋅(I₁ - I₂))

In [50]:
T_sym = adcs_sym.eq_torquefree_motion(3)
T1_sym = T_sym[0]
T2_sym = T_sym[1]
T3_sym = T_sym[2]

T1_sym, T2_sym, T3_sym


(I₁⋅\dot{\omega}₁ - I₂⋅\omega₂⋅\omega₃ + \omega₂⋅(I₃⋅\omega₃ + hₛ), I₁⋅\omega₁
⋅\omega₃ + I₂⋅\dot{\omega}₂ - \omega₁⋅(I₃⋅\omega₃ + hₛ), -I₁⋅\omega₁⋅\omega₂ +
 I₂⋅\omega₁⋅\omega₂ + I₃⋅\dot{\omega}₃)

From first condition for stability:

In [51]:
# Substitute values for:
    # inertia = I1, I2
    # dot angular velocity x and y = dotomega1sim, dotomega2sim = 0, 0 <- the spacecraft is not rotating in the x, y directions. Only in z direction
T1_sym = T1_sym.subs(omega3sim,wz).subs(I2_sym, I2).subs(I3_sym, I3).subs(I1_sym, I1).subs(dotomega1sim, 0).subs(dotomega2sim, 0).simplify()
T2_sym = T2_sym.subs(omega3sim, wz).subs(I2_sym, I2).subs(I3_sym, I3).subs(I1_sym, I1).subs(dotomega1sim, 0).subs(dotomega2sim, 0).simplify()

(0 > T1_sym), (0 > T2_sym)


(\omega₂⋅(hₛ - 0.2) < 0, -\omega₁⋅(hₛ + 0.2) < 0)

In [52]:
solve(T1_sym, hs_sym), solve(T2_sym, hs_sym)

([0.2], [-0.2])

From first condition for stability:

$h_s \gt 0.2$ N.m.s, $\hspace 1em$ $h_s \gt -0.2$ N.m.s.

Therfore, for passive stability we take the largest equivalence and hence: $h_s \gt 0.2$ N.m.s

-------------------

From second condition for stability:

Using the same T1_sym and T2_sym as before:

$h_s \lt 0.2$ N.m.s, $\hspace 1em$ $h_s \lt -0.2$ N.m.s.

Therfore, for passive stability we take the smallest equivalence and hence: $h_s \lt -0.2$ N.m.s